In [8]:
import tensorflow as tf
import numpy as np

default_model = ""
samsung_model = "./yolo_v4_tiny_float/samsung_yolo_v4_tiny_float.tflite"

In [9]:
model_path = samsung_model

interpreter = tf.lite.Interpreter(model_path=model_path)

interpreter.allocate_tensors()

In [10]:
# 입력 및 출력 정보 얻기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 입력 텐서 정보 출력
print("== input tensor information ==")
for i, detail in enumerate(input_details):
    print(f"input tensor {i}:")
    print(f"\tname: {detail['name']}")
    print(f"\tshape: {detail['shape']}")
    print(f"\tdtype: {detail['dtype']}")
    print()

# 출력 텐서 정보 출력
print("== output tensor information ==")
for i, detail in enumerate(output_details):
    print(f"output tensor {i}:")
    print(f"\tname: {detail['name']}")
    print(f"\tshape: {detail['shape']}")
    print(f"\tdtype: {detail['dtype']}")
    print()

== input tensor information ==
input tensor 0:
	name: input_1
	shape: [  1 416 416   3]
	dtype: <class 'numpy.float32'>

== output tensor information ==
output tensor 0:
	name: tf_op_layer_concat_15/concat_15
	shape: [   1 2535    4]
	dtype: <class 'numpy.float32'>

output tensor 1:
	name: tf_op_layer_concat_16/concat_16
	shape: [   1 2535   80]
	dtype: <class 'numpy.float32'>



In [23]:
from PIL import Image
import numpy as np

image_path = "./images/bird-8047149_1280.jpg"

image = Image.open(image_path)
image = image.resize((416, 416))
input_data = np.array(image, dtype=np.float32)
input_data = input_data / 255.0
input_data = np.expand_dims(input_data, axis=0)
# input_data = np.expand_dims(input_data / 255.0, axis=0)

In [22]:
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_boxes = interpreter.get_tensor(output_details[0]['index'])
output_classes = interpreter.get_tensor(output_details[1]['index'])
output_scores = interpreter.get_tensor(output_details[2]['index'])
output_data = [output_boxes, output_scores, output_classes]

IndexError: list index out of range

In [36]:
# 이미지 전처리 함수
def preprocess_image(image_path, input_size):
    image = Image.open(image_path)
    image = image.resize(([input_size[1], input_size[0]]))
    image = np.array(image, dtype=np.float32)
    image = image / 255.0  # YOLO는 일반적으로 [0, 1] 범위로 정규화
    image = np.expand_dims(image, axis=0)  # 배치 차원 추가
    return image

In [47]:
import cv2

# 이미지 후처리 함수
def process_output(output_data, input_image, threshold=0.5):
    boxes, classes = output_data
    boxes = boxes[0]
    # scores = scores[0]
    classes = classes[0]
    
    # 입력 이미지 크기 가져오기
    input_h, input_w, _ = input_image.shape

    for i in range(len(scores)):
        if scores[i] > threshold:
            # 바운딩 박스 그리기
            ymin, xmin, ymax, xmax = boxes[i]
            xmin = int(xmin * input_w)
            xmax = int(xmax * input_w)
            ymin = int(ymin * input_h)
            ymax = int(ymax * input_h)
            
            cv2.rectangle(input_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            label = f"{int(classes[i])}: {scores[i]:.2f}"
            cv2.putText(input_image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return input_image

In [46]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# YOLOv4 TFLite 모델 경로
model_path = samsung_model

# 모델 불러오기
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# 입력 및 출력 텐서 정보 얻기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 모델 입력 크기 (보통 YOLOv4는 416x416 또는 608x608)
input_shape = input_details[0]['shape'][1:3]

print(input_shape)
# 이미지 전처리
input_data = preprocess_image(image_path, input_shape)

# 입력 텐서 설정
interpreter.set_tensor(input_details[0]['index'], input_data)

# 추론 실행
interpreter.invoke()

# 출력 텐서 가져오기
output_boxes = interpreter.get_tensor(output_details[0]['index'])
output_classes = interpreter.get_tensor(output_details[1]['index'])
output_data = [output_boxes, output_classes]

# 원본 이미지 로드                              
original_image = cv2.imread(image_path)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# 감지된 객체 후처리 및 시각화
result_image = process_output(output_data, original_image_rgb)

# 결과 이미지 출력
plt.figure(figsize=(10, 10))
plt.imshow(result_image)
plt.axis('off')
plt.show()

original_image = cv2.imread(image_path)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

result_image = process_output(output_data, original_image_rgb)

[416 416]


UnboundLocalError: local variable 'scores' referenced before assignment